In [ ]:
import numpy as np
import scipy.signal as sig
import bokeh.plotting as bkp
import bokeh.models as bkm
bkp.output_notebook()

from asl_bloch_sim import bloch, rf_design

In [ ]:
# PCASL: A typical real sequence may use 750 0.5 ms, 20º, Hann RF pulses over a 1500 ms period
duration = 2.5 # seconds
label_duration = 1.5 # seconds
num_reps = 750
dt = 0.00001 # seconds

flip_angle = 20 # degrees
rf_duration = 0.0005 # seconds
rf_bandwidth = 125 # Hz
off_resonance = 1000 # Hz
spectrum_lines = 400

G_avg = 2 # mT/m

T1 = 1.5 # seconds
T2 = 0.2 # seconds

In [ ]:
time = np.arange(0, duration, dt) # seconds
rf_time = np.arange(-rf_duration / 2, rf_duration / 2, dt)

rf_pulse = rf_design.sinc_pulse(flip_angle, rf_duration, rf_bandwidth, dt, phase_angle=0)

rf_label = np.tile(rf_design.extend(rf_pulse, label_duration / num_reps, dt), num_reps)
rf_control = np.tile(np.append(rf_design.extend(rf_pulse, label_duration / num_reps, dt),
                               rf_design.extend(rf_pulse * -1, label_duration / num_reps, dt)), num_reps // 2)
rf = rf_design.extend(rf_control, duration, dt)

dfz = np.linspace(0, off_resonance, spectrum_lines) # Hz
B = bloch.construct_B_field(rf, off_resonance=dfz)

In [ ]:
# plot RF with bokeh
plot = bkp.figure(width=800, height=400, title='RF pulse')
plot.line(rf_time * 1e3, rf_pulse.real * 1e6, line_width=2)
plot.line(rf_time * 1e3, rf_pulse.imag * 1e6, line_width=2, color='orange')
plot.xaxis.axis_label = 'Time (ms)'
plot.yaxis.axis_label = 'RF Amplitude (µT)'
bkp.show(plot)

In [ ]:
freq = np.fft.fftshift(np.fft.fftfreq(50000, dt))
amp = np.log10(np.abs(np.fft.fftshift(np.fft.fft(np.append(rf_design.extend(rf_pulse, label_duration / num_reps, dt),
                               rf_design.extend(rf_pulse * -1, label_duration / num_reps, dt)).real, n=50000))) / 1e-6) * 20
# plot RF with bokeh
plot = bkp.figure(width=800, height=400, title='RF pulse')
plot.line(freq, amp, line_width=2)
plot.xaxis.axis_label = 'Frequency (Hz)'
plot.yaxis.axis_label = 'RF Amplitude (µT)'
bkp.show(plot)

In [ ]:
# plot RF with bokeh
plot = bkp.figure(width=800, height=400, title='RF pulses')
plot.line(time, rf.real * 1e6, line_width=2, alpha=0.5)
plot.line(time, rf.imag * 1e6, line_width=2, color='orange', alpha=0.5)
plot.xaxis.axis_label = 'Time (s)'
plot.yaxis.axis_label = 'RF Amplitude (µT)'
bkp.show(plot)

In [ ]:
mag = np.array([[0, 0, 1]]) # initial magnetization
mags = np.array([mag := bloch.relax(bloch.precess(mag, B[step], dt), T1, T2, dt) for step in range(round(duration / dt))])

In [ ]:
# plot magnetization with bokeh
plot = bkp.figure(width=800, height=400, title='Magnetization')
plot.line(time, mags[:, 0, 0], line_width=2, legend_label='Mx', alpha=0.5)
plot.line(time, mags[:, 0, 1], line_width=2, legend_label='My', color='orange', alpha=0.5)
plot.line(time, mags[:, 0, 2], line_width=2, legend_label='Mz', color='green')
plot.xaxis.axis_label = 'Time (s)'
plot.yaxis.axis_label = 'Magnetization (a.u.)'
plot.x_range = bkm.DataRange1d(start=0, end=1.6)
bkp.show(plot)

In [ ]:
# plot magnetization off-resonances with bokeh
title = 'Longitudinal Magnetization with Off-Resonance Pulse'
plot = bkp.figure(width=1000, height=500, title=title)
for offres in range(0, end := mags.shape[1], end // 10):
    alpha = 1 - offres / end
    plot.line(time, mags[:, offres, 2], line_width=2, legend_label=f'{dfz[offres]:g} Hz',
              alpha=alpha, color='green')
plot.xaxis.axis_label = 'Time (s)'
plot.yaxis.axis_label = 'Magnetization (a.u.)'
plot.x_range = bkm.DataRange1d(start=0, end=1.5)
bkp.show(plot)

In [ ]:
# flipped = np.min(mags[..., 2], axis=0)
flipped = np.argmin(mags[..., 2], axis=0)
plot = bkp.figure(width=800, height=400, title='Flipped Magnetization Spectrum')
plot.line(dfz, np.take_along_axis(mags[..., 2], flipped[np.newaxis], axis=0)[0], line_width=2)
plot.line(dfz, time[flipped], line_width=2)
plot.xaxis.axis_label = 'Off-Resonance Frequency (Hz)'
plot.yaxis.axis_label = 'Magnetization (a.u.)'
plot.y_range = bkm.DataRange1d(start=-1, end=1)
bkp.show(plot)

In [ ]:
avg_long_mag = np.mean(mags[:round(label_duration/dt), ..., 2], axis=0)
plot = bkp.figure(width=800, height=400, title='Average Longitudinal Magnetization Spectrum of Control Sequence')
plot.line(dfz, avg_long_mag, line_width=2)
plot.xaxis.axis_label = 'Off-Resonance Frequency (Hz)'
plot.yaxis.axis_label = 'Magnetization (a.u.)'
plot.y_range = bkm.DataRange1d(start=-1, end=1)
bkp.show(plot)

In [ ]:
# save mags, dt, and dfz to compressed numpy file
np.savez_compressed('mags.npz', mags=mags[::10], dt=dt * 10, dfz=dfz)